In [3]:
from datascience import *
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import os

In [4]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
SPREADSHEET_ID = '1waiCSO7hm8kmWLaG8OdB8_7ksfEbm_B3CyKHbXTIf5E'
RANGE_NAME = 'AY 22-23!A2:F'
dateColumn = []
timeColumn = []
itemBarcodeColumn = []
quantityColumn = []
itemNameColumn = []
remainingColumn = []
def getSheetInfo():
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('sheets', 'v4', credentials=creds)

        # Call the Sheets API
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                    range=RANGE_NAME).execute()
        values = result.get('values', [])

        if not values:
            print('No data found.')
            return
        for row in values:
            dateColumn.append(row[0])
            timeColumn.append(row[1])
            itemBarcodeColumn.append(row[2])
            quantityColumn.append(int(row[3]))
            itemNameColumn.append(row[4])
            remainingColumn.append(int(row[5]))
    except HttpError as err:
        print(err)

getSheetInfo()

In [5]:
checkout = Table().with_columns("Date", dateColumn,"Time", timeColumn, "Item Barcode", itemBarcodeColumn, "Quantity", quantityColumn, "Item Name", itemNameColumn, "Remaining #", remainingColumn)
checkout

Date,Time,Item Barcode,Quantity,Item Name,Remaining #
8/18/2022 12:15:55,12:15,027862020120,9,Firm Tofu,0
8/18/2022 12:16:52,12:16,027862020168,10,Soft Tofu,0
8/18/2022 12:17:13,12:17,029243050351,1,Macaroni,0
8/22/2022 15:57:00,15:57,024182025064,1,Edensoy Soy Milk,47
,,apple,1,Apple,-1
9/8/2022 10:01:45,10:01,754686002437,1,"Almond Original, Unsweetened Almond Milk",35
,,027000500064,1,Hunt's Pasta Sauce,154
,,apple juice,1,"Totally Juice, 100% Apple Juice",33
,,035200056427,1,Brown Rice,177
9/8/2022 10:02:27,10:02,apple juice,1,"Totally Juice, 100% Apple Juice",32
